In [1]:
import math
from scipy.stats import linregress
from scipy.fft import fftfreq
import pandas as pd
import h5py
import numpy as np

lc = pd.read_csv('AP31396872.csv', usecols= ['hjd','mag', 'mag_err'])

In [2]:
class EstadisticasLuzEstelar:
    """
    Clase para calcular estadísticas básicas de una serie temporal de magnitudes de luz estelar.

    Parámetros:
    - lc:array
        Lista que contiene las magnitudes de luz estelar ('mag').

    Atributos:
    - lista_ordenada: list
        Lista ordenada de las magnitudes.
    - promedio: float
        Promedio de las magnitudes.
    - mediana: float
        Mediana de las magnitudes.
    - varianza: float
        Varianza de las magnitudes.
    """

    def __init__(self, lc):
        self.lc = lc
        self.lista_ordenada = sorted(lc['mag'])
        self.promedio = self.calcular_promedio()
        self.mediana = self.calcular_mediana()
        self.varianza = self.calcular_varianza()

    def calcular_promedio(self):
        """Calcula el promedio de las magnitudes."""
        suma = sum(self.lc['mag'])
        return suma / len(self.lc['mag'])

    def calcular_mediana(self):
        """Calcula la mediana de las magnitudes."""
        cant = len(self.lista_ordenada)
        if cant % 2 == 0:
            return (self.lista_ordenada[cant // 2 - 1] + self.lista_ordenada[cant // 2]) / 2
        else:
            return self.lista_ordenada[cant // 2]

    def calcular_varianza(self):
        """Calcula la varianza de las magnitudes."""
        suma = sum([(i - self.promedio)**2 for i in self.lc['mag']])
        return suma / len(self.lc['mag'])

class EstadisticasLuzEstelarExtendido(EstadisticasLuzEstelar):
    """
    Clase que extiende EstadisticasLuzEstelar para calcular estadísticas adicionales.

    Atributos adicionales:
    - amplitud: float
        Amplitud de las magnitudes.
    - kurtosis: float
        Kurtosis de las magnitudes.
    - autocorrelacion: float
        Coeficiente de autocorrelación de las magnitudes.
    - asimetria: float
        Asimetría de las magnitudes.
    - desviacion_estandar: float
        Desviación estándar de las magnitudes.
    - Percentiles: float
        percentiles 20, 35, 50, 65, 80 de las magnitudes
    """

    def __init__(self, lc):
        super().__init__(lc)
        self.amplitud = self.calcular_amplitud()
        self.kurtosis = self.calcular_kurtosis()
        self.autocorrelacion = self.calcular_autocorrelacion()
        self.asimetria = self.calcular_asimetria()
        self.desviacion_estandar = self.calcular_desviacion_estandar()
        self.percentiles = self.percentiles()

    def calcular_amplitud(self):
        """Calcula la amplitud de las magnitudes."""
        return self.lista_ordenada[-1] - self.mediana

    def calcular_kurtosis(self):
        """Calcula la kurtosis de las magnitudes."""
        suma = sum([(i - self.promedio)**4 for i in self.lc['mag']])
        suma2 = sum([(i - self.promedio)**2 for i in self.lc['mag']])
        N = len(self.lc['mag'])
        return ((1/N) * suma) / ((suma2/N)**2) - 3

    def calcular_autocorrelacion(self, lag=1):
        """Calcula el coeficiente de autocorrelación de las magnitudes."""
        lista = self.lc['mag']
        n = len(lista)
        suma = sum([(lista[i] - self.promedio) * (lista[i + lag] - self.promedio) for i in range(n - lag)])
        suma2 = sum([(lista[i] - self.promedio)**2 for i in range(n - lag)])
        return suma / suma2

    def calcular_asimetria(self):
        """Calcula la asimetría de las magnitudes."""
        lista = self.lc['mag']
        n = len(lista)
        suma = sum([(lista[i] - self.promedio)**3 for i in range(n - 1)])
        suma2 = sum([(lista[i] - self.promedio)**2 for i in range(n - 1)])
        return (suma / n) / (suma2 / n)**(3/2)

    def calcular_desviacion_estandar(self):
        """Calcula la desviación estándar de las magnitudes."""
        return math.sqrt(self.varianza)
    
    def percentiles():
        """Calcula los percentiles 20 35 50 65 y 80 de las magnitudes"""
        
        lista = self.lc['mag']
    
        p20_index =  int(0.2 * (len(lista) + 1)) - 1
        p20 = lista[p20_index]
    
        p35_index = int(0.35* (len(lista) + 1)) - 1
        p35 = lista[p35_index]
    
        p50_index = int(0.5* (len(lista) + 1)) - 1
        p50 = lista[p50_index]
    
        p65_index = int(0.65* (len(lista) + 1)) - 1
        p65 = lista[p65_index]
    
        p80_index = int(0.8* (len(lista) + 1)) - 1
        p80 = lista[p80_index]
    
        return p20, p35, p50, p65, p80
    
    
    

class AnalisisFrecuencias:
    """
    Clase para realizar análisis de frecuencias en una serie temporal de magnitudes de luz estelar.

    Parámetros:
    - lc: array
        Arreglo que contiene las magnitudes de luz estelar ('mag') y los tiempos ('hjd').

    Atributos:
    - frecuencia_fundamental: float
        Frecuencia fundamental de la serie temporal.
    """

    def __init__(self, lc):
        self.lc = lc
        self.frecuencia_fundamental = self.calcular_frecuencia_fundamental()

    def calcular_frecuencia_fundamental(self):
        """Calcula la frecuencia fundamental de la serie temporal."""
        tiempo = self.lc['hjd']
        flujo = self.lc['mag']
        N = len(tiempo)
        fs = 1 / (tiempo[1] - tiempo[0])
        frequencies = fftfreq(N, 1/fs)
        positive_frequencies = frequencies[:N//2]
        
        # Calcula el espectro de potencia
        power_spectrum = np.abs(np.fft.fft(flujo)[:N//2]) ** 2

        # Encuentra la frecuencia fundamental como la frecuencia con mayor potencia
        index_fundamental = np.argmax(power_spectrum[1:]) + 1
        frecuencia_fundamental = positive_frequencies[index_fundamental]

        return frecuencia_fundamental    
    
    
    def calcular_frecuencias_armonicos(self):
        """Calcula las frecuencias de los armónicos."""
        tiempo = self.lc['hjd']
        num_armonicos = 4
        fs = 1 / (tiempo[1] - tiempo[0])
        frequencies = fftfreq(len(tiempo), 1/fs)
        positive_frequencies = frequencies[:len(frequencies)//2]
        return positive_frequencies[1:num_armonicos+1]

    def calcular_periodo(self):
        """Calcula el período asociado a la frecuencia fundamental."""
        return 1 / self.frecuencia_fundamental

    def calcular_periodograma(self):
        """Calcula el periodograma de la serie temporal."""
        tiempo = self.lc['hjd']
        flujo = self.lc['mag']
        N = len(tiempo)
        dt = tiempo[1] - tiempo[0]
        frecuencias = [1 / (N * dt) * i for i in range(N)]
        potencia = [(sum([fl * math.cos(2 * math.pi * f * t) for t, fl in zip(tiempo, flujo)])**2 +
                     sum([fl * math.sin(2 * math.pi * f * t) for t, fl in zip(tiempo, flujo)])**2) / N
                    for f in frecuencias]
        return frecuencias, potencia


def guardar_en_hdf5(estadisticas, estadisticas_extendido, analisis_frecuencias, archivo_salida='resultados.h5'):
    """Guarda las estadísticas y frecuencias en un archivo HDF5."""
    with h5py.File(archivo_salida, 'a') as archivo_hdf5:
        # Grupo para EstadisticasLuzEstelar
        grupo_estadisticas_basico = archivo_hdf5.create_group('EstadisticasLuzEstelar')
        grupo_estadisticas_basico.attrs['Estadistica básica'] = 'Estadísticas básicas de una serie temporal de magnitudes de luz estelar'
        
        # Crear conjuntos de datos
        grupo_estadisticas_basico.create_dataset('promedio', data=estadisticas.promedio)
        grupo_estadisticas_basico.create_dataset('mediana', data=estadisticas.mediana)
        grupo_estadisticas_basico.create_dataset('varianza', data=estadisticas.varianza)
        
        # Agregar atributos (descripciones) después de crear los conjuntos de datos
        grupo_estadisticas_basico['promedio'].attrs['Promedio'] = 'Promedio de las magnitudes'
        grupo_estadisticas_basico['mediana'].attrs['Mediana'] = 'Mediana de las magnitudes'
        grupo_estadisticas_basico['varianza'].attrs['Varianza'] = 'Varianza de las magnitudes'

        # Grupo para EstadisticasLuzEstelarExtendido
        grupo_estadisticas_extendido = archivo_hdf5.create_group('EstadisticasLuzEstelarExtendido')
        grupo_estadisticas_extendido.attrs['Estadistica extendida'] = 'Estadísticas extendidas de una serie temporal de magnitudes de luz estelar'
        
        # Crear conjuntos de datos
        grupo_estadisticas_extendido.create_dataset('amplitud', data=estadisticas_extendido.amplitud)
        grupo_estadisticas_extendido.create_dataset('kurtosis', data=estadisticas_extendido.kurtosis)
        grupo_estadisticas_extendido.create_dataset('autocorrelacion', data=estadisticas_extendido.autocorrelacion)
        grupo_estadisticas_extendido.create_dataset('asimetria', data=estadisticas_extendido.asimetria)
        grupo_estadisticas_extendido.create_dataset('desviacion_estandar', data=estadisticas_extendido.desviacion_estandar)
        grupo_estadisticas_extendido.create_dataset('percentiles', data=estadisticas_extendido.percentiles)


        # Agregar atributos después de crear los conjuntos de datos
        grupo_estadisticas_extendido['amplitud'].attrs['Amplitud'] = 'Amplitud de las magnitudes'
        grupo_estadisticas_extendido['kurtosis'].attrs['Kurtosis'] = 'Kurtosis de las magnitudes'
        grupo_estadisticas_extendido['autocorrelacion'].attrs['Autocorelacion'] = 'Coeficiente de autocorrelación de las magnitudes'
        grupo_estadisticas_extendido['asimetria'].attrs['Asimetria'] = 'Asimetría de las magnitudes'
        grupo_estadisticas_extendido['desviacion_estandar'].attrs['Desviacirn estandas'] = 'Desviación estándar de las magnitudes'

        # Grupo para AnalisisFrecuencias
        grupo_frecuencias = archivo_hdf5.create_group('AnalisisFrecuencias')
        grupo_frecuencias.attrs['Analisis curvas'] = 'Análisis de frecuencias en una serie temporal de magnitudes de luz estelar'
        
        # Crear conjuntos de datos
        grupo_frecuencias.create_dataset('frecuencia_fundamental', data=analisis_frecuencias.frecuencia_fundamental)
        grupo_frecuencias.create_dataset('frecuencias_armonicos', data=analisis_frecuencias.calcular_frecuencias_armonicos())
        grupo_frecuencias.create_dataset('periodo_asociado', data=analisis_frecuencias.calcular_periodo())
        frecuencias, potencia = analisis_frecuencias.calcular_periodograma()
        grupo_frecuencias.create_dataset('frecuencias_periodograma', data=frecuencias)
        grupo_frecuencias.create_dataset('potencia_periodograma', data=potencia)

        # Agregar atributos después de crear los conjuntos de datos
        grupo_frecuencias['frecuencia_fundamental'].attrs['Frecuencia fundamental'] = 'Frecuencia fundamental de la serie temporal'
        grupo_frecuencias['frecuencias_armonicos'].attrs['Frecuencia armonicos'] = 'Frecuencias de los armónicos'
        grupo_frecuencias['periodo_asociado'].attrs['Periodo'] = 'Período asociado a la frecuencia fundamental'
        grupo_frecuencias['frecuencias_periodograma'].attrs['Frecuencia periodograma'] = 'Frecuencias del periodograma'
        grupo_frecuencias['potencia_periodograma'].attrs['Potencia periodograma'] = 'Potencia del periodograma'

    


estadisticas =  EstadisticasLuzEstelar(lc)
estadisticas_extendido = EstadisticasLuzEstelarExtendido(lc)
analisis_frecuencias = AnalisisFrecuencias(lc)
guardar_en_hdf5(estadisticas, estadisticas_extendido, analisis_frecuencias)


TypeError: EstadisticasLuzEstelarExtendido.percentiles() takes 0 positional arguments but 1 was given